# Proyecto práctico: árbol de decisión y random forest con scikit-learn

In [253]:
#Importamos las librerias principales
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 

Utilizaremos el **Car Evaluation Data Set** de Kaggle: https://www.kaggle.com/datasets/elikplim/car-evaluation-data-set

In [254]:
#Cargamos dataset a utilizar
df_car = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data', header=None)
df_car.head()

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


## Análisis exploratorio de datos

In [255]:
#Analizamos el shape del objeto
df_car.shape

(1728, 7)

In [256]:
#Como podemos observar vamos a renombrar las columnas de manera correcta
col_names = ['buyin', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']

df_car.columns = col_names

df_car.head()

,buyin,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [257]:
#Visualizamos los tipos de datos
df_car.dtypes

buyin       object
maint       object
doors       object
persons     object
lug_boot    object
safety      object
class       object
dtype: object

Primer resumen de los datos:
* Hay 7 variables en el conjunto de datos. Todas las variables son de tipo de datos categóricos.
* Estos se dan por compra, mantenimiento, puertas, personas, lug_boot, seguridad y clase.
* La clase es la variable de destino o target.

In [258]:
# Exploremos un poco mas la variable target
df_car['class'].value_counts()

unacc    1210
acc       384
good       69
vgood      65
Name: class, dtype: int64

In [259]:
#Verificamos valores missings
df_car.isnull().sum()

buyin       0
maint       0
doors       0
persons     0
lug_boot    0
safety      0
class       0
dtype: int64

El dataset no contiene valores faltantes

## Procesamiento de datos

In [260]:
#Separamos en X e y

# Variables predictoras
X = df_car.drop(['class'], axis=1)

# Variable target
y = df_car['class']

In [261]:
# Variables features
X.head()

,buyin,maint,doors,persons,lug_boot,safety
0,vhigh,vhigh,2,2,small,low
1,vhigh,vhigh,2,2,small,med
2,vhigh,vhigh,2,2,small,high
3,vhigh,vhigh,2,2,med,low
4,vhigh,vhigh,2,2,med,med


In [262]:
# Variable predictora
y.head()

0    unacc
1    unacc
2    unacc
3    unacc
4    unacc
Name: class, dtype: object

In [263]:
#Importamos las librerias necesarias para la creacion del modelo
from sklearn.model_selection import train_test_split

#30% para test y 70% para train
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30, random_state=42)

El 42 es una "semilla". Su principal razón es para que si quieres replicar tu código, te salgan los mismos resultados. ¿Por qué? Porque cuando separa tus datos entre datos de entrenamiento y datos de prueba, lo hace de una manera aleatoria, entonces si vuelves a ejecutar el código sin una semilla, cada vez te separará los datos de maneras distintas. En cambio, todas las veces que lo haga con la semilla "42", lo hará igual. Entonces el número de la semilla es irrelevante, podría ser cualquiera que tú quieras (00000, 42, 3, etc.), lo importante es que si usas el mismo número, tendrás los mismos resultados

In [264]:
#Veamos que obtuvimos
X_train.shape, X_test.shape

((1209, 6), (519, 6))

In [265]:
y_train.shape, y_test.shape

((1209,), (519,))

In [266]:
#Veamos que tenemos. Por ejemplo, en X_train
X_train.head()

,buyin,maint,doors,persons,lug_boot,safety
1178,med,med,5more,4,big,high
585,high,high,3,more,small,low
1552,low,med,3,4,med,med
1169,med,med,5more,2,big,high
1033,med,high,4,2,big,med


In [267]:
X_train.dtypes

buyin       object
maint       object
doors       object
persons     object
lug_boot    object
safety      object
dtype: object

## Entrenamiento de modelo de clasificación con árbol de decisión

Para convertir variables object a categóricas, puedes usar cualquiera de las tres opciones:

- **OrdinalEncoder** es una buena opción si tus variables tienen un orden natural. Por ejemplo, si tienes una variable “tamaño” con valores “pequeño”, “mediano” y “grande”, puedes usar OrdinalEncoder para asignar valores numéricos a cada uno de ellos en función de su orden natural.

- **OneHotEncoder** es una buena opción si tus variables no tienen un orden natural. Por ejemplo, si tienes una variable “color” con valores “rojo”, “verde” y “azul”, no hay un orden natural entre ellos. En este caso, OneHotEncoder creará una columna separada para cada valor posible y asignará un valor binario (0 o 1) a cada columna.

- **category_encoder** es otra biblioteca que puedes usar para codificar variables categóricas. Tiene varias opciones diferentes para codificar variables categóricas, incluyendo OrdinalEncoder , OneHotEncoder, BinaryEncoder, BaseNEncoder, HashingEncoder.

### Convertir con category_encoder

In [268]:
#Importante: todos nuestros tipos de datos son object, realizamos una transformacion con category_enconder
import category_encoders as ce  # debemos instalar esta libreria: pip install category_encoders

# Convertimos las variables predictoras
encoder = ce.OrdinalEncoder(cols=X_train.columns)

# Aplicamos las variables transformadas a los datos de entrenamiento
X_train1 = encoder.fit_transform(X_train)

# Aplicamos las variables transformadas a los datos de prueba
X_test1 = encoder.transform(X_test)


fit_transform y transform son dos métodos que se utilizan en el contexto de la transformación de datos en aprendizaje automático. En particular, cuando hablamos de OrdinalEncoder, estos métodos tienen funciones específicas:

**fit_transform**: Este método se utiliza para ajustar el codificador a los datos de entrada y transformarlos simultáneamente. Durante el ajuste (fit), el codificador aprende los diferentes valores únicos presentes en la característica y asigna a cada uno un número entero único. Luego, estos valores se utilizan para transformar los datos de entrada, es decir, cada valor único se reemplaza por su correspondiente número entero asignado.

**transform**: Este método se utiliza únicamente para transformar los datos de entrada utilizando las asignaciones aprendidas durante el ajuste (fit). Si intentas usar transform sin haber ajustado el codificador previamente (sin haber llamado fit), obtendrás un error.

Entonces, la diferencia principal radica en que fit_transform realiza tanto el ajuste como la transformación de los datos, mientras que transform solo realiza la transformación, utilizando los parámetros aprendidos durante el ajuste.

In [269]:
#Verificamos la transformacion
X_train1.head()

,buyin,maint,doors,persons,lug_boot,safety
1178,1,1,1,1,1,1
585,2,2,2,2,2,2
1552,3,1,2,1,3,3
1169,1,1,1,3,1,1
1033,1,2,3,3,1,3


In [270]:
X_test1.head()

,buyin,maint,doors,persons,lug_boot,safety
599,2,2,3,3,3,1
1201,1,4,4,1,3,3
628,2,2,1,3,1,3
1498,3,2,1,1,3,3
1263,1,4,3,2,3,2


### Convertir con Ordinal Encoder 

Otra forma de transformar los Datos a Numericos...

In [273]:
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()

X_train2 = pd.DataFrame(ord_enc.fit_transform(X_train), columns=X_train.columns)

X_test2 = pd.DataFrame(ord_enc.transform(X_test), columns=X_test.columns)

In [274]:
X_train2.head()

,buyin,maint,doors,persons,lug_boot,safety
0,2.0,2.0,3.0,1.0,0.0,0.0
1,0.0,0.0,1.0,2.0,2.0,1.0
2,1.0,2.0,1.0,1.0,1.0,2.0
3,2.0,2.0,3.0,0.0,0.0,0.0
4,2.0,0.0,2.0,0.0,0.0,2.0


In [275]:
X_test2.head()

,buyin,maint,doors,persons,lug_boot,safety
0,0.0,0.0,2.0,0.0,1.0,0.0
1,2.0,1.0,0.0,1.0,1.0,2.0
2,0.0,0.0,3.0,0.0,0.0,2.0
3,1.0,0.0,3.0,1.0,1.0,2.0
4,2.0,1.0,2.0,2.0,1.0,1.0


### Entrenamiento del modelo

In [271]:
#Importar árbol de decisión
from sklearn.tree import DecisionTreeClassifier

#Creacion del modelo
tree = DecisionTreeClassifier(max_depth=2, random_state=42)

Hemos definido un nivel de profundidad de 2 debido a que tenemos pocos datos. Recordar que este algortimo a medida que crece en profundidad tiende al overfitting. Con cada 5000 registros podemos aumentar la profundidad en una unidad

In [277]:
#Entrenamiento
tree.fit(X_train1, y_train)

DecisionTreeClassifier(max_depth=2, random_state=42)

In [279]:
#Calculo de las predicciones en Train y Test
y_train_pred_tree =  tree.predict(X_train1)
y_test_pred_tree =  tree.predict(X_test1)

In [280]:
# mostramos las predicciones
y_train_pred_tree

array(['acc', 'unacc', 'unacc', ..., 'acc', 'unacc', 'unacc'],
      dtype=object)

## Evaluación de modelo de clasificación con árbol de decisión

In [ ]:
#Calculo de metricas 


#Calculo el accuracy en Train


#Calculo el accuracy en Test


In [ ]:
#Verificamos el feature importances


## Entrenamiento de modelo de clasificación con random forest

In [ ]:
#Importar random forest


In [ ]:
#Calculo de las predicciones en Train y Test


## Evaluación de modelo de clasificación con random forest

In [ ]:
#Calculo de metricas 


#Calculo el accuracy en Train


#Calculo el accuracy en Test


#Importante: podriamos reducir el numero de estimadores para disminuir el sobreajuste del modelo.

In [ ]:
# Visualizacion de las feature importantes


In [ ]:
#Grafico de barras


In [ ]:
# Matriz de confusion del RF


In [ ]:
#RF
